##### Imports

In [3]:
# Add PyDatAnalysis to path
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, "/Users/owensheekey/Documents/Research/PyDatAnalysis")

export_path = 'Exports/'

In [410]:
from __future__ import annotations
from progressbar import progressbar
from src.DatObject.Make_Dat import get_dat, get_dats
import src.UsefulFunctions as U
from src.DataStandardize.ExpSpecific.Feb21 import Feb21Exp2HDF, Feb21ExpConfig
from src.DataStandardize.ExpConfig import ExpConfigGroupDatAttribute, ExpConfigBase
import multiprocessing as mp
import plotly.graph_objs as go
import numpy as np
import lmfit as lm
from typing import TYPE_CHECKING, Iterable, Optional
from src.DatObject.Attributes.Transition import i_sense_digamma, i_sense, i_sense_digamma_quad
from src.UsefulFunctions import edit_params
from src.DatObject.Attributes.SquareEntropy import square_wave_time_array, integrate_entropy
import logging
logger = logging.getLogger(__name__)
import src.UsefulFunctions as U
import scipy.io
from scipy.interpolate import RectBivariateSpline
import matplotlib.pyplot as plt

##### Narrow fit

In [368]:
def narrow_fit(dat, width, **kwargs):
    '''
    Get a fit only including +/- width in dat.x around center of transition
    kwargs is the stuff to pass to get_fit
    Return a fit
    '''
    out = dat.SquareEntropy.get_Outputs(existing_only=True)
    x = np.copy(out.x)
    y = np.copy(out.averaged)
    y = np.mean(y[(0, 2), :], axis=0)

    start_ind = np.nanargmin(np.abs(np.add(x, width)))
    end_ind = np.nanargmin(np.abs(np.subtract(x, width)))

    x[:start_ind] = [np.nan] * start_ind
    x[end_ind:] = [np.nan] * (len(x) - end_ind)

    y[:start_ind] = [np.nan] * start_ind
    y[end_ind:] = [np.nan] * (len(y) - end_ind)

    fit = dat.SquareEntropy.get_fit(
        x=x,
        data=y,
        **kwargs)
    return fit

In [6]:
def narrow_fit_trans_only(dat, width, **kwargs):
    '''
    Get a fit only including +/- width in dat.x around center of transition
    kwargs is the stuff to pass to get_fit
    Return a fit
    '''
    x = np.copy(dat.Transition.avg_x)
    y = np.copy(dat.Transition.avg_data)

    start_ind = np.nanargmin(np.abs(np.add(x, width)))
    end_ind = np.nanargmin(np.abs(np.subtract(x, width)))

    x[:start_ind] = [np.nan] * start_ind
    x[end_ind:] = [np.nan] * (len(x) - end_ind)

    y[:start_ind] = [np.nan] * start_ind
    y[end_ind:] = [np.nan] * (len(y) - end_ind)

    fit = dat.SquareEntropy.get_fit(
        x=x,
        data=y,
        **kwargs)
    return fit

##### DoCalc, getDeltaT

In [272]:
def do_calc(datnum, overwrite=True):
    """Just a function which can be passed to a process pool for faster calculation"""
    save_name = 'SPS.01'

    dat = get_dat(datnum)

    setpoints = [0.01, None]

    # Get other inputs
    setpoint_times = square_wave_time_array(dat.SquareEntropy.square_awg)
    sp_start, sp_fin = [U.get_data_index(setpoint_times, sp) for sp in setpoints]
    logger.debug(f'Setpoint times: {setpoints}, Setpoint indexs: {sp_start, sp_fin}')

    # Run Fits
    pp = dat.SquareEntropy.get_ProcessParams(name=None,  # Load default and modify from there
                                             setpoint_start=sp_start, setpoint_fin=sp_fin,
                                             transition_fit_func=i_sense,
                                             save_name=save_name)
    out = dat.SquareEntropy.get_Outputs(name=save_name, inputs=None, process_params=pp, overwrite=overwrite)
    dat.Entropy.get_fit(which='avg', name=save_name, data=out.average_entropy_signal, x=out.x, check_exists=False,
                        overwrite=overwrite)
    [dat.Entropy.get_fit(which='row', row=i, name=save_name,
                         data=row, x=out.x, check_exists=False,
                         overwrite=overwrite) for i, row in enumerate(out.entropy_signal)]
    return out

In [8]:
def get_deltaT(dat):
    """Returns deltaT of a given dat in mV"""
    ho1 = dat.AWG.max(0)  # 'HO1/10M' gives nA * 10
    t = dat.Logs.temps.mc

    # Datnums to search through (only thing that should be changed)
    datnums = set(range(1312, 1451+1)) - set(range(1312, 1451+1, 4))
    # datnums = set()
    # for j in range(5):
    #     datnums = datnums.union(set(range(28 * j + 1312, 28 * j + 1312 + 4 * 7 + 1)) - set([28 * j + 1312 + 4 * i for i in range(8)]))
    # datnums = list(datnums)

    dats = get_dats(datnums)

    dats = [d for d in dats if np.isclose(d.Logs.temps.mc, dat.Logs.temps.mc, rtol=0.1)]  # Get all dats where MC temp is within 10%
    bias_lookup = np.array([d.Logs.fds['HO1/10M'] for d in dats])

    indp = np.argmin(abs(bias_lookup - ho1))
    indm = np.argmin(abs(bias_lookup + ho1))
    theta_z = np.nanmean([d.Transition.avg_fit.best_values.theta for d in dats if d.Logs.fds['HO1/10M'] == 0])

    # temp_lookup = np.array([d.Logs.temps.mc for d in dats])
    # bias_lookup = np.array([d.Logs.fds['HO1/10M'] for d in dats])
    #
    # indp = np.argmin(temp_lookup - t + bias_lookup - ho1)
    # indm = np.argmin(temp_lookup - t + bias_lookup + ho1)
    # indz = np.argmin(temp_lookup - t + bias_lookup)

    theta_p = dats[indp].Transition.avg_fit.best_values.theta
    theta_m = dats[indm].Transition.avg_fit.best_values.theta
    # theta_z = dats[indz].Transition.avg_fit.best_values.theta
    return (theta_p + theta_m) / 2 - theta_z

##### NRG interps

In [458]:
NRG = scipy.io.loadmat('Results.mat')
occ = NRG["Occupation_mat"]
ens = np.reshape(NRG["Ens"], 401)
ts  = np.reshape(NRG["Ts"], 70)
ens = np.flip(ens)
occ = np.flip(occ, 0)
interp = RectBivariateSpline(ens, np.log10(ts), occ, kx=1,ky=1)
def interpNRG(x, logt, dx=1, amp=1, center=0, lin=0, const=0):
    ens = np.multiply(np.add(x,center), dx)
    curr = [interp(en, logt)[0][0] for en in ens]
    scaled_current = np.multiply(curr, amp)
    scaled_current += const + np.multiply(lin,x)
    return scaled_current

In [456]:
NRG = scipy.io.loadmat('Results.mat')
intdndt = NRG["intDNDT_mat"]
ens = np.reshape(NRG["Ens"], 401)
ts  = np.reshape(NRG["Ts"], 70)
ens = np.flip(ens)
intdndt = np.flip(intdndt, 0)
interp_intdndt = RectBivariateSpline(ens, np.log10(ts), intdndt, kx=1,ky=1)
def interpNRG_intdndt(x, logt, dx=1, amp=1, center=0):
    ens = np.multiply(np.add(x,center),dx)
    val = [interp_intdndt(en, logt)[0][0] for en in ens]
    val = np.multiply(val, amp)
    return val

In [461]:
tstemp  = np.reshape(NRG["Ts"], 70)
xx = np.linspace(-100,100,201)
fig = go.Figure()
for i in range(57):
    fig.add_trace(go.Scatter(mode='markers', x=xx, y=interpNRG(xx, np.log10(tstemp[i]), dx = tstemp[i]), name=f'g/t={0.001/tstemp[i]:.2f}'))



fig.update_layout(xaxis_title='Ens*T', yaxis_title='Occupation /arb')
fig.show(renderer="browser")

In [462]:
tstemp  = np.reshape(NRG["Ts"], 70)
xx = np.linspace(-100,100,201)
fig = go.Figure()
for i in range(57):
    fig.add_trace(go.Scatter(mode='markers', x=xx, y=interpNRG_intdndt(xx, np.log10(tstemp[i]), dx = tstemp[i]), name=f'g/t={0.001/tstemp[i]:.2f}'))



fig.update_layout(xaxis_title="Ens*T", yaxis_title='Entropy /kb')
fig.show(renderer="browser")

In [281]:
# datnums = set(range(1869, 1919)) - set(range(1870, 1919, 2))
# transdatnums = set(range(1869, 1919)) - set(range(1869, 1919, 2))
datnums = np.sort(list(set(range(2082, 2089))))# - set(range(2016, 2065, 2))))
#transdatnums = np.sort(list(set(range(2015, 2065)) - set(range(2015, 2065, 2))))
datnums = np.array([2164, 2167, 2170])
transdatnums = np.add(datnums, 1)

In [282]:
dats = get_dats(list(datnums), overwrite=False)
transdats = get_dats(list(transdatnums), overwrite=False)

In [276]:
biass = [dat.AWG.max(0) for dat in dats]
datnums = [dat.datnum for dat in dats]

In [277]:
out = [do_calc(dn, overwrite=False) for dn in progressbar(datnums)]

100% (3 of 3) |##########################| Elapsed Time: 0:00:19 Time:  0:00:19


In [319]:
Θ = 3.9
fit = transdats[0].Transition.get_fit(which='avg', check_exists=False)
params = fit.params
params.add('g', value=0, vary=True, min=-50, max=1000)
new_pars = edit_params(params, param_name='theta', value=Θ, vary=False) 

In [405]:
nrg_pars = lm.Parameters()
nrg_pars.add_many(
    ('dx',  0.0027313, False, None, None, None, None),
    ('amp', 2, True, 0.1, None, None, None),
    ('center', 0, True, None, None, None, None),
    ('lin', 0.1, True, None, None, None, None),
    ('const', 0, True, None, None, None, None),
    ('logt', -2, True, -3.5, -3, None, None))

In [406]:
nrg_fit_ = [narrow_fit_trans_only(
    dat,
    600,
    which='avg', 
    initial_params=nrg_pars, 
    fit_func=interpNRG, 
    check_exists=False,
    overwrite=True)
for dat in progressbar(transdats)]

100% (3 of 3) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [408]:
nrg_fit_[1].best_values

dx=0.0027313
amp=0.44114
center=1.0758
lin=0.0010149
const=7.1233
logt=-3

In [409]:
fig = go.Figure()
for i, dat in progressbar(enumerate(transdats[:5])):
    x = dat.Transition.avg_x
    y = dat.Transition.avg_data
    fit = nrg_fit_[i]
    xfit = np.linspace(-400,400,1001)
    yfit = fit.eval_fit(xfit)
    fig.add_trace(go.Scatter(mode='markers', x=x[::10], y=y[::10], name=f'{datnums[i]}d'))
    fig.add_trace(go.Scatter(mode='lines', x=xfit, y=yfit, name=f'{datnums[i]}f', marker_color='grey'))
fig.update_layout(xaxis_title='ACC/100 /mV', yaxis_title='Current /nA',
                      title=f'Dat {dats[0].datnum} - {dats[-1].datnum}')
fig.show(renderer="browser")

| |#                                                  | 2 Elapsed Time: 0:00:00


In [363]:
fig.write_html(f'Exports/NRGFits_thermally_broadened_03_17_dats{datnums[0]}_{datnums[-1]}.html')

In [328]:
''' Plot any NRG linecuts '''
fig = go.Figure()
xfit = np.linspace(-400,400,1001)
yfit = interpNRG(xfit, -0.00046476, 0.97871, 3.4102, 0.0015914, 6.7055, 0)
fig.add_trace(go.Scatter(mode='markers', x=xfit, y=yfit, name=f'{datnums[i]}f', marker_color='grey'))

fig.update_layout(xaxis_title='ACC/100 /mV', yaxis_title='Current /nA',
                      title=f'Dat {dats[0].datnum} - {dats[-1].datnum}')
fig.show(renderer="browser")

In [369]:
nrg_fit_cold = [narrow_fit(
    dat,
    600,
    initial_params=nrg_pars, 
    fit_func=interpNRG, 
    check_exists=False, overwrite=True)
for dat in progressbar(dats)]

100% (3 of 3) |##########################| Elapsed Time: 0:00:03 Time:  0:00:03


In [263]:
fig = go.Figure()
for i, dat in progressbar(enumerate(dats[:5])):
    out = dat.SquareEntropy.get_Outputs(existing_only=True)
    x = np.copy(out.x)
    y = np.copy(out.averaged)
    y = np.mean(y[(0,2), :], axis=0)
    fit = nrg_fit_cold[i]
    xfit = np.linspace(-100,100,1001)
    yfit = fit.eval_fit(xfit)
    fig.add_trace(go.Scatter(mode='markers', x=x, y=y, name=f'{datnums[i]}d'))
    fig.add_trace(go.Scatter(mode='lines', x=xfit, y=yfit, name=f'{datnums[i]}f', marker_color='grey'))
fig.update_layout(xaxis_title='ACC/100 /mV', yaxis_title='Current /nA',
                      title=f'Dat {dats[0].datnum} - {dats[-1].datnum}')
fig.show(renderer="browser")

| |#                                                  | 4 Elapsed Time: 0:00:00


In [131]:
fig.write_html(f'Exports/NRGFits_cold_thermally_broadened_03_12_dats{datnums[0]}_{datnums[-1]}.html')

In [370]:
nrg_fit_cold[0].best_values

dx=0.0027387
amp=0.94071
center=-2.1321
lin=0.0015091
const=6.7254
logt=-2

In [140]:
dats[0].SquareEntropy.get_Outputs(existing_only=True).average_entropy_signal
dats[0].SquareEntropy.get_Outputs(existing_only=True).x

In [152]:
nrgmodel_dndt = lm.Model(interpNRG_dndt)
nrg_pars_dndt = lm.Parameters()
nrg_pars_dndt.add_many(
    ('dx', 0.0026624, False, None, None, None, None),
    ('amp', 1, True, None, None, None, None),
    ('center', 0.00583, False, None, None, None, None),
    ('logt', -2, False, None, None, None, None))

In [208]:
result = nrgmodel_dndt.fit(dats[0].SquareEntropy.get_Outputs(existing_only=True).average_entropy_signal, nrg_pars, x=dats[0].SquareEntropy.get_Outputs(existing_only=True).x)

ValueError: NaN values detected in your input data or the output of your objective/model function - fitting algorithms cannot handle this! Please read https://lmfit.github.io/lmfit-py/faq.html#i-get-errors-from-nan-in-my-fit-what-can-i-do for more information.

In [382]:
fig = go.Figure()
amps = [0.0025, 0.001, 0.001]
for i, dat in progressbar(enumerate(dats)):
    out = dat.SquareEntropy.get_Outputs(existing_only=True)
    x = np.copy(out.x)
    y = np.copy(out.average_entropy_signal)
    xfit = np.linspace(-100,100,1001)
    yfit = interpNRG_intdndt(xfit, nrg_fit_cold[i].best_values.dx, amps[i], nrg_fit_cold[i].best_values.center, nrg_fit_cold[i].best_values.logt)
    fig.add_trace(go.Scatter(mode='markers', x=x, y=y, name=f'{datnums[i]}d, {biass[i]}'))
    fig.add_trace(go.Scatter(mode='lines', x=xfit, y=yfit, name=f'{datnums[i]}f', marker_color='grey'))
fig.update_layout(xaxis_title='ACC/100 /mV', yaxis_title='Current /nA',
                      title=f'Dat {dats[0].datnum} - {dats[-1].datnum}')
fig.show(renderer="browser")

| |#                                                  | 2 Elapsed Time: 0:00:00


In [265]:
fig.write_html(f'Exports/NRGFits_dndt_hot{datnums[0]}-{datnums[-1]}.html')

In [31]:
fig = go.Figure()

fig.add_trace(go.Scatter(mode='markers', x=escs, y=amp_digamma_cold, text=datnums, name="Entropy fits"))
fig.add_trace(go.Scatter(mode='markers', x=escs, y=amp_digamma_, text=transdatnums, name="Transition fits"))

fig.update_layout(xaxis_title='ESC/ mV', yaxis_title='Amplitude /nA',
                      title=f'Dats {dats[0].datnum} - {dats[-1].datnum}')
fig.show(renderer="browser")

In [32]:
fig.write_html(f'Exports/Ampls_03_12_dats{datnums[0]}_{datnums[-1]}.html')

In [33]:
fig = go.Figure()

fig.add_trace(go.Scatter(mode='markers', x=escs, y=np.array(g_digamma_cold)/Θ, text=datnums, name="Entropy fits"))
fig.add_trace(go.Scatter(mode='markers', x=escs, y=np.array(g_digamma_)/Θ, text=transdatnums, name="Transition fits"))
fig.update_layout(xaxis_title='ESC/ mV', yaxis_title='Gamma/Theta /arb',
                      title=f'Dats {dats[0].datnum} - {dats[-1].datnum}')

fig.show(renderer="browser")

In [34]:
fig.write_html(f'Exports/Gammas_03_12_dats{datnums[0]}_{datnums[-1]}.html')

In [35]:
fig = go.Figure()
for i, dat in progressbar(enumerate(dats)):
    x = dat.SquareEntropy.avg_x[::10]
    y = dat.SquareEntropy.default_Output.averaged
    ycold = np.mean(y[(0, 2), :], axis=0)[::10]
    yhot = np.mean(y[(1, 3), :], axis=0)[::10]
    xfit = np.linspace(-1000,1000,1001)
    fit = narrow_fit(
            dat,
            600,
            initial_params=new_pars, 
            fit_func=i_sense_digamma, 
            check_exists=False)
    yfit = fit.eval_fit(xfit) - fit.best_values.lin*xfit
    fig.add_trace(go.Scatter(mode='markers', x=x, y=ycold - fit.best_values.lin*x, name=f'{datnums[i]}d_cold'))
    fig.add_trace(go.Scatter(mode='markers', x=x, y=yhot - fit.best_values.lin*x, name=f'{datnums[i]}d_hot'))
    fig.add_trace(go.Scatter(mode='lines', x=xfit, y=yfit, name=f'{datnums[i]}f', marker_color='grey'))
fig.update_layout(xaxis_title='ACC/100 /mV', yaxis_title='Current /nA',
                      title=f'Dat {dats[0].datnum} - {dats[-1].datnum}')
fig.show(renderer="browser")

| |                #                                 | 24 Elapsed Time: 0:00:01


In [36]:
fig.write_html(f'Exports/Cold_hot_transitions_plus_fit_dats{datnums[0]}_{datnums[-1]}.html')

In [39]:
deltaT = [get_deltaT(dat) for dat in dats]
ampl = amp_digamma_
for i, dat in enumerate(dats):
    dat.Entropy.set_integration_info(dT=deltaT[i], amp=ampl[i], overwrite=True)

In [40]:
fig = go.Figure()
int_ents = []
for i in range(len(dats)):
    int_ent = integrate_entropy(out[i].average_entropy_signal, dats[i].Entropy.integration_info.sf)
    int_ents.append(int_ent[-1])
    fig.add_trace(go.Scatter(mode='markers', 
                             x=np.subtract(out[i].x, mids_digamma_[i]), 
                             y=int_ent,
                             name= f'g/t:{np.divide(g_digamma_,Θ)[i]:.2f}, dat{dats[i].datnum}, ESS:{dats[i].Logs.fds["ESS"]}'))

fig.add_trace(go.Scatter(mode='lines', x=[-400,400], y=[np.log(2), np.log(2)], name="Log2"))  
fig.add_trace(go.Scatter(mode='lines', x=[-400,400], y=[np.log(3), np.log(3)], name="Log3"))  
fig.update_layout(xaxis_title='ACC/100 /mV', yaxis_title='Entropy /kb',
                      title=f'Dats {dats[0].datnum} - {dats[-1].datnum}')
fig.show(renderer='browser')

In [41]:
fig.write_html(f'Exports/IntEntropy_03_12_dats{datnums[0]}_{datnums[-1]}.html')

In [363]:
# escs1 = []
# escs2 = []
# int_ents1 = []
# int_ents2 = []
# datnums1 = []
# datnums2 = []
# for i in range(len(escs)):
#     if dats[i].datnum >= 1941:
#         escs2.append(escs[i])
#         int_ents2.append(int_ents[i])
#         datnums2.append(dats[i].datnum)
#     else:
#         escs1.append(escs[i])
#         int_ents1.append(int_ents[i])
#         datnums1.append(dats[i].datnum)

In [44]:
fig = go.Figure()
fig.add_trace(go.Scatter(mode='markers+lines', 
                         x=escs, y=int_ents, 
                         text=[dat.datnum for dat in dats], 
                         name="Int"))
# fig.add_trace(go.Scatter(mode='markers+lines', 
#                          x=escs1, y=int_ents1, 
#                          text=datnums1, 
#                          name="Set 1 -- 10 nA"))
# fig.add_trace(go.Scatter(mode='markers+lines', 
#                          x=escs2, y=int_ents2, 
#                          text=datnums2, 
#                          name="Set 2 -- 10 nA"))
# #                          marker=dict(color='LightSkyBlue',
#                                      size=10,
#                                      line=dict(color='DarkSlateGrey',
#                                                width=2)),
#                          error_y=dict(type='data', # value of error bar given in data coordinates
#                                       array=[0.05]*len(int_ents),
#                                       visible=True)))
# fig.add_trace(go.Scatter(mode='lines', x=[-500,0], y=[np.log(2), np.log(2)], name="Log2", line=dict(color='DarkSlateGrey', width=4, dash='dot')))  

fig.update_layout(xaxis_title='ESC /mV', yaxis_title='Entropy /kb',
                      title=f'Integrated -- Dats {dats[0].datnum} - {dats[-1].datnum}')
fig.show(renderer="browser")

In [45]:
fig.write_html(f'Exports/IntEntropyvalues_03_12_dats{datnums[0]}_{datnums[-1]}.html')